In [67]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re,string,unicodedata
import nltk
import warnings
warnings.filterwarnings('ignore')
import spacy
nlp = spacy.load("en_core_web_sm")
from sklearn.naive_bayes import GaussianNB
# from sklearn.svm import SVC
# from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.metrics import accuracy_score ,roc_auc_score,f1_score,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.preprocessing import FunctionTransformer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('popular')
nltk.download('wordnet')



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\naren\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\naren\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\naren\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\naren\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\naren\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\nar

True

In [68]:
df_train=pd.read_csv('train.csv',encoding='latin-1')

In [69]:
df_train.head()
#first 5 rows

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26


In [70]:
df_train=df_train[['text','sentiment']]

In [71]:
df_train['text'] = df_train['text'].astype(str)

In [72]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       27481 non-null  object
 1   sentiment  27481 non-null  object
dtypes: object(2)
memory usage: 429.5+ KB


In [73]:
#df_train=df_train[df_train['sentiment']!='neutral']
df_train['sentiment']=df_train['sentiment'].map({'positive':1,'negative':0,'neutral':2})

In [74]:
X=df_train['text']
y=df_train['sentiment']

In [75]:
class TextCleaner():
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if type(X) == str:
            return self.clean_text(X)
        else:
            return [self.clean_text(text) for text in X]
    def clean_text(self, text):
        text = str(text).lower()  # Make text lowercase
        text = re.sub('\[.*?\]', '', text)  # Remove any sequence of characters in square brackets
        text = re.sub('https?://\S+|www\.\S+', '', text)  # Remove links
        text = re.sub('<.*?>+', '', text)  # Remove HTML tags
        text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  # Remove punctuation
        text = re.sub('\n', '', text)  # Remove newline characters
        text = re.sub('\w*\d\w*', '', text)  # Remove words containing numbers
        text = re.sub(r'[^a-z/A-Z/0-9/ ]', '', text)  # Remove special characters
        return text

In [76]:
class stopwords():
    
    def fit(self,X,y=None):
        return self
    
    def transform(self, X):
        if type(X) == str:
            return self.stopw(X)
        else:
            return [self.stopw(text) for text in X]
    
    def stopw(self,text):
        from nltk.corpus import stopwords
        stopwords = stopwords.words('english')
        stopwords=stopwords+['s','m','u','im','ye','id','atg','na','ta','gon','wan']
        text= ' '.join([x for x in text.split() if x not in stopwords])
        return text

In [77]:
class lemma:
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if type(X) == str:
            return [self.lemmatize(X)]
        else:
            return [self.lemmatize(text) for text in X]

    def lemmatize(self, text):
        tokens = word_tokenize(text)
        lemmatized_tokens = [self.lemmatizer.lemmatize(token) for token in tokens]
        lemmatized_text = " ".join(lemmatized_tokens)
        return lemmatized_text

In [78]:
# class lemma():
    
#     def __init__(self,lemma_model):
#         self.lemma_model=lemma_model
        
#     def fit(self,X,y=None):
#         return self
    
#     def transform(self,X):
#         if type(X) == str:
#             return [self.lemmatise(X)]
#         else:
#             return [self.lemmatise(text) for text in X]
    
#     def lemmatise(self,text):
#         return " ".join([token.lemma_ for token in self.lemma_model(text)])

In [79]:
def to_array(X):
    return X.toarray()
    

In [80]:
pipeline = Pipeline([
    ('cleaner', TextCleaner()),
    ('stop_words_removal',stopwords()),
    ('lemmatization',lemma())
#     ('TF-IDF-fit',TfidfVectorizer()),
    #('TF-IDF-Transform',FunctionTransformer(to_array,accept_sparse=True))
    #('toarray',FunctionTransformer(to_array,accept_sparse=True)),
#     ('model',MultinomialNB())
    
        
])

In [81]:
# pipe=pipeline.fit(X,y)

In [82]:
tf=TfidfVectorizer()
X_tfidf=tf.fit_transform(X)

In [83]:
mnb=MultinomialNB()
mnb_model=mnb.fit(X_tfidf,y)

In [84]:
# with gzip.open('path/to/your_model.pkl.gz', 'wb') as file:
#     pickle.dump(your_data, file)

In [85]:
import pickle
import gzip
pickle.dump(pipeline, gzip.open('Text_preprocessing_3.pkl', 'wb'))
pickle.dump(tf,gzip.open('vectorizer_3.pkl','wb'))
pickle.dump(mnb,gzip.open('Sentiment_detector_3.pkl','wb'))

In [51]:
txt_clean=pickle.load(gzip.open('Text_preprocessing_3.pkl','rb'))
tf=pickle.load(gzip.open('vectorizer_3.pkl','rb'))
model=pickle.load(gzip.open('Sentiment_detector_3.pkl','rb'))

In [52]:
X=txt_clean.fit_transform(X)

In [53]:
X=tf.transform(X)

In [54]:
pred=model.predict(X)

In [55]:
accuracy_score(y,pred)

0.767948764600997

In [56]:
df_test=pd.read_csv('test.csv',encoding='latin-1')

In [57]:
df_test=df_test[['text','sentiment']]

In [58]:
df_test['text'] = df_test['text'].astype(str)

In [59]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4815 entries, 0 to 4814
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       4815 non-null   object
 1   sentiment  3534 non-null   object
dtypes: object(2)
memory usage: 75.4+ KB


In [60]:
#df_test=df_test[df_test['sentiment']!='neutral']
df_test['sentiment']=df_test['sentiment'].map({'positive':1,'negative':0,'neutral':2})

In [61]:
X_test=df_train['text']
y_test=df_train['sentiment']

In [62]:
X_test=txt_clean.fit_transform(X_test)

In [63]:
X_test=tf.transform(X_test)

In [64]:
pred_test=model.predict(X_test)

In [66]:
f1_score(y_test,pred_test,average='weighted')

0.7667793489048255